# config

In [ ]:
# 超参配置

class Hyperparameter:
    # ################################################################
    #                             Data
    # ################################################################
    device = 'cuda'
    data_root = './data/'
    cifar10_root = '../input/cifar10-python/cifar-10-batches-py'
    train_image_root = './data/train_images'
    test_image_root = './data/test_images'

    class_num = 10
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    seed = 1234  # random seed

    # ################################################################
    #                             Experiment
    # ################################################################
    batch_size = 32
    init_lr = 0.01
    epochs = 20
    verbose_step = 20
    save_step = 10000


HP = Hyperparameter()


# preprocess

In [ ]:
import random
import os
import json

random.seed(HP.seed)

for foldername in ['data', 'log', 'model_save', HP.train_image_root, HP.test_image_root]:
    if not os.path.exists(foldername):
        os.mkdir(foldername)


def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

cls2id = {}
for id, cls in enumerate(HP.class_names):
    cls2id[cls] = id
json.dump(cls2id, open('./data/cls2id.json', 'w'))

import glob
import numpy as np
import cv2

for batch_name, image_root in [('/data_batch*', HP.train_image_root), ('/test_batch*', HP.test_image_root)]:
    data_batch_files = HP.cifar10_root + batch_name
    train_list = glob.glob(data_batch_files)

    for l in train_list:
        l_dict = unpickle(l)

        for im_idx, im_data in enumerate(l_dict[b'data']):
            im_label = l_dict[b'labels'][im_idx]
            im_name = l_dict[b'filenames'][im_idx]

            im_label_name = HP.class_names[im_label]
            im_data = np.reshape(im_data, [3, 32, 32])
            im_data = np.transpose(im_data, (1, 2, 0))

            if not os.path.exists('{}/{}'.format(image_root, im_label_name)):
                os.mkdir('{}/{}'.format(image_root, im_label_name))

            cv2.imwrite('{}/{}/{}'.format(image_root, im_label_name, im_name.decode('utf-8')), im_data)
print('图片数据初始化完毕')

# dataset_cifar10

In [ ]:
import glob
import os.path

import torch
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms as T
import json

cls2id = json.load(open('./data/cls2id.json', 'r'))

default_transform = T.Compose([
    T.ToTensor()
])


def default_loader(path):
    return Image.open(path)


class CifarDataset(torch.utils.data.Dataset):
    def __init__(self, im_list, transform=default_transform, loader=default_loader):
        imgs = []

        for im_item in im_list:
            im_label_name = im_item.split('/')[-2]
            imgs.append((im_item, cls2id[im_label_name]))

        self.imgs = imgs
        self.transform = transform
        self.loader = loader

    def __getitem__(self, idx):
        im_path, im_label, = self.imgs[idx]
        im_data = self.loader(im_path)

        return self.transform(im_data).to(HP.device), im_label

    def __len__(self):
        return len(self.imgs)

# model

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F


class ClassiNet(nn.Module):

    def __init__(self):
        super(ClassiNet, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.max_pooling1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.conv2_1 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.conv2_2 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.max_pooling2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.conv3_1 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        self.conv3_2 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        self.max_pooling3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=1)

        self.conv4_1 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.conv4_2 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.max_pooling4 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=512 * 2 * 2, out_features=HP.class_num)
        )

    def forward(self, x):
        batchsize = x.size(0)
        out = self.conv1(x)
        out = self.max_pooling1(out)

        out = self.conv2_1(out)
        out = self.conv2_2(out)
        out = self.max_pooling2(out)

        out = self.conv3_1(out)
        out = self.conv3_2(out)
        out = self.max_pooling3(out)

        out = self.conv4_1(out)
        out = self.conv4_2(out)
        out = self.max_pooling4(out)

        out = out.view(batchsize, -1)

        out = self.fc(out)
        out = F.log_softmax(out, dim=1)

        return out

# trainer

In [ ]:
import os.path
import random
import torch
import numpy as np
from tensorboardX import SummaryWriter
from argparse import ArgumentParser
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import glob
from torchvision import transforms as T

logger = SummaryWriter('./log')

# seed init: 保证模型的可复现性
torch.manual_seed(HP.seed)
random.seed(HP.seed)
np.random.seed(HP.seed)
torch.cuda.manual_seed(HP.seed)


def evaluate(model, devloader, crit):
    model.eval()
    sum_loss = 0.
    with torch.no_grad():
        for batch in devloader:
            x, y = batch
            pred = model(x)
            loss = crit(pred, y.to(HP.device))
            sum_loss += loss.item()

    model.train()
    return sum_loss / len(devloader)


def save_checkpoint(model, epoch, opt, save_path):
    save_dict = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': opt.state_dict()
    }
    torch.save(save_dict, save_path)

train_transform = T.Compose([
    T.RandomResizedCrop((28, 28)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(90),
    T.RandomGrayscale(0.1),
    T.ColorJitter(0.3, 0.3, 0.3, 0.3),
    T.ToTensor()
])

test_transform = T.Compose([
    T.Resize((28, 28)),
    T.ToTensor()
])

def train():
    model = ClassiNet().to(HP.device)

    criterion = nn.CrossEntropyLoss()

    opt = optim.Adam(model.parameters(), lr=HP.init_lr)

    im_train_list = glob.glob('./data/train_images/*/*.png')
    im_test_list = glob.glob('./data/test_images/*/*.png')

    trainset = CifarDataset(im_train_list, transform=train_transform)
    train_loader = DataLoader(trainset, batch_size=HP.batch_size, shuffle=True, drop_last=True)

    devset = CifarDataset(im_test_list, transform=test_transform)
    dev_loader = DataLoader(devset, batch_size=HP.batch_size, shuffle=True, drop_last=False)

    start_epoch, step = 0, 0

    model.train()

    for epoch in range(start_epoch, HP.epochs):
        print('Start Epoch: %d, Steps: %d' % (epoch, len(train_loader)))
        for batch in train_loader:
            x, y = batch  # 加载数据
            opt.zero_grad()  # 梯度归零
            pred = model(x)
            loss = criterion(pred, y.to(HP.device))

            loss.backward()
            opt.step()

            logger.add_scalar('Loss/Train', loss, step)

            if not step % HP.verbose_step:
                eval_loss = evaluate(model, dev_loader, criterion)
                logger.add_scalar('Loss/Dev', eval_loss, step)

            if not step % HP.save_step:
                model_path = 'model_%d_%d.model' % (epoch, step)
                save_checkpoint(model, epoch, opt, os.path.join('model_save', model_path))

            step += 1
            logger.flush()
            print('Epoch:[%d/%d], step:%d, Train Loss:%.5f, Dev Loss:%.5f' % (
                epoch, HP.epochs, step, loss.item(), eval_loss))

    logger.close()

# 训练

In [ ]:
train()

In [ ]:
!tar -zcvf log.tar.gz ./log

from IPython.display import FileLink
FileLink('log.tar.gz')